In [1]:
import cv2
import requests
import json
from itertools import repeat

In [2]:
# Configure to look for faces

cascPath = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# Create the haar cascade

faceCascade = cv2.CascadeClassifier(cascPath)

In [3]:
# Analyze an image for faces and count the results
# (drawing on may examples out there, e.g. https://realpython.com/face-recognition-with-python/) 

def count_faces(imagePath):
    # Read the image
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    return len(faces)

In [4]:
# Define some Tropy API functions

# Set API endpoint
api_url_base = 'http://localhost:2019'

# Get all Tropy items from the current project
def get_all_items():   
    call = '%s/project/items/' % api_url_base
    res = requests.get(call)
    if res.status_code != 200:
        return
    items_dict = json.loads(res.text)
    itemIDs = [ k['id'] for k in items_dict]
    return itemIDs

# Get a Tropy item's list of photos
def get_photos(itemID):
    call = '%s/project/items/%s/photos' % (api_url_base, itemID)
    res = requests.get(call) 
    if res.status_code != 200:
        return
    item_dict = json.loads(res.text)
    photos = [ k['path'] for k in item_dict]
    return photos

# Tag a Tropy item by tag names
# Warning: API currently only supports a single tag
def tag_item_by_tag_name(itemID, tag_names):
    tag_IDs = []
    for tag_name in tag_names:
        tag_IDs.append(create_tag(tag_name))
    params = list(zip(repeat('tag'), tag_IDs))
    call = '%s/project/items/%s/tags' % (api_url_base, itemID)
    res = requests.post(call, data=params)
    return

# Tag a Tropy item by tag ID
# Warning: API currently only supports a single tag
def tag_item_by_tag_ID(itemID, tag_IDs):
    params = list(zip(repeat('tag'), tag_IDs))
    call = '%s/project/items/%s/tags' % (api_url_base, itemID)
    res = requests.post(call, data=params)
    return

# Create a Tropy tag
def create_tag(name, color=''):
    # if tag already exists return existing tag ID
    tags = get_tags()
    for tag in tags:
        if tag['name'].lower() == name.lower():
            return tag['id']
    params = [('name', name), ('color', color)]
    call = '%s/project/tags' % api_url_base
    res = requests.post(call, data=params)
    if res.status_code != 200:
        return
    tag_json = json.loads(res.text)
    return tag_json['id']
    

# Get a Tropy tag by its name
def get_tag_by_name(name):
    tags = get_tags()
    tag_id = None
    for tag in tags:
        if tag['name'].lower() == name.lower():
            tag_id = tag['id']
            return get_tag_by_id(tag_id)
        else:
            break

# Get a Tropy tag by its ID
def get_tag_by_id(tag_id):
    call = '%s/project/tags/%s' % (api_url_base, tag_id)
    res = requests.get(call)
    if res.status_code != 200:
        return
    return json.loads(res.text)
    
# Get all Tropy tags
def get_tags():
    call = '%s/project/tags' % api_url_base
    res = requests.get(call)
    if res.status_code != 200:
        return
    return json.loads(res.text)


In [5]:
# Get the images a from each Tropy item in list of items
# Since we are only interested in groups of people, we'll set a threshold of faces to ignore false positives
# If item has any photo above the threshold, tag item as "People"

items = get_all_items()
threshold = 5
tags = ['People']

for item in items:
    photos = get_photos(item)
    for photo in photos:
        faces = count_faces(photo)
        if faces > threshold:
            tag_item_by_tag_name(item, tags)
            break